# Options for encoding ordinal response

- Naive categorical encoding (ignore order)
- Integer encoding (really regression not classification)
     - For numpy array, use sklearn's LabelEncoder (string -> integer)
     - For pandas DataFrame, can try OrdinalEncoder in the [Category Encoders package](http://contrib.scikit-learn.org/categorical-encoding/index.html)
- [Ordinal crossentropy loss for Keras](https://github.com/JHart96/keras_ordinal_categorical_crossentropy)
- [Ordinal regression for TF](https://github.com/gspell/TF-OrdinalRegression)
- "Cumulative" encoding [Cheng et al.]
     - [Creating custom encoders](https://towardsdatascience.com/custom-transformers-and-ml-data-pipelines-with-python-20ea2a7adb65)
- Split into K-1 binary classification problems [Frank and Hall] 
     - not sure if it's efficient with neural nets
     - Cheng et al.'s encoding does this in some sense, within a single network

In [1]:
# set up
# if installed, keras uses tf as backend
import keras
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, RMSprop
from keras.optimizers import Adagrad, Adadelta, Adam, Adamax, Nadam

from sklearn import model_selection
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV
from sklearn.model_selection import cross_val_score

import category_encoders as ce

# Import ordinal crossentropy loss function
import sys
sys.path.insert(0, ".")
from keras_ordinal_categorical_crossentropy import ordinal_categorical_crossentropy as OCC

Using TensorFlow backend.


## Simulated ordinal response

Let's generate toy data for which we actually know the 
data generating process (DGP). This will provide a better
benchmark for the different approaches.

In [2]:
# 0. set rng seed
np.random.seed(123)

# 1. set parameters

K = 3 # response categories
N = 10000 # number of examples
P = 3 # number of features

# thresholds:
mu0 = 0
mu1 = 3.14

# set DGP parameters
b0 = 1
b1 = 2
b2 = -2
b3 = 1

In [3]:
# 2. generate features
# TODO: auto-generate x_mean and x_cov
# TODO: generate categorical features
x_mean = (1, 2, 0.5) 
x_cov = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
X = np.random.multivariate_normal(x_mean, x_cov, N) # dim: NxK
# X.shape

In [4]:
# 3. generate normal error
u = np.random.normal(size=(N,1)) # dim: Nx1
# u.shape

In [5]:
# 4. generate latent response
B = np.array([[b1], [b2], [b3]]) # dim: Kx1
y_latent = b0 + X.dot(B) + u # dim: Nx1
# y_latent.shape

In [6]:
[np.min(y_latent), np.mean(y_latent), np.median(y_latent), np.max(y_latent)]

[-13.413067400889629,
 -0.4856418920579942,
 -0.4903702239051473,
 11.038336261288451]

In [7]:
# 5. generate ordinal response
y = np.digitize(y_latent, [mu0, mu1], right=1)
# y.shape

In [8]:
np.asarray(np.unique(y, return_counts=True))

array([[   0,    1,    2],
       [5622, 3149, 1229]])

In [9]:
# 6. one-hot encoding
Y = keras.utils.to_categorical(y, num_classes=K)

In [10]:
# create train / test split
train_x, test_x, train_y, test_y = model_selection.train_test_split(X,y,test_size = 0.1, random_state = 0)

# encode response as categorical
train_y_cat = keras.utils.to_categorical(train_y, num_classes=K)
test_y_cat = keras.utils.to_categorical(test_y, num_classes=K)

In [ ]:
# parameters for k-fold CV
n_folds = 10 # default is 5

## Hyperparameters to test

- number of layers: more layers not better?
- other types of layers (LeakyReLu, PReLu, ELU, ThresholdedReLU):
- number of nodes: problem specific? more nodes not better...
- activations: tanh, sigmoid slightly better than relu. others?
- dropout: smaller dropout slightly better?
- other regularization: activation reg not good; 
- initializer (kernel, bias)
- optimizer
- epochs
- batch

Let's begin

### 1. Categorical encoding

Compare categorical cross-entropy vs ordinal cross-entropy

In [13]:
# Function to create model
def create_model(optimizer='sgd', init='glorot_uniform',,
                 n_input=64, n_hidden=64, dropout=0.5,
                 p=1, k=1, input_act='relu', output_act='softmax',
                 loss='categorical_crossentropy'):
    # create model
    model = Sequential()
    model.add(Dense(n_input, activation=input_act, input_dim=p, kernel_initializer=init))
    model.add(Dropout(dropout))
    model.add(Dense(n_hidden, activation=input_act, kernel_initializer=init))
    model.add(Dropout(dropout))
    model.add(Dense(k, activation=output_act, kernel_initializer=init))
    # Compile model
    model.compile(loss=loss, 
                  optimizer=optimizer, 
                  metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, p=P, k=K, verbose=0)

In [14]:
# grid search epochs, batch size, initializer, and optimizer
## loss = ['categorical_crossentropy', 'ordinal_crossentropy']
## optimizers = ['sgd', 'rmsprop', 'adagrad', 'adadelta', 'adam', 'adamax', 'nadam']
## inits = ['glorot_uniform', 'glorot_normal', 'uniform', 'normal']
optimizers = ['sgd', 'rmsprop']
inits = ['glorot_uniform', 'glorot_normal']
epochs = [20, 200]
batches = [16, 128]
param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=inits)

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=n_folds)

# one-hot encoding
grid_result = grid.fit(X, Y)

In [15]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.849800 using {'batch_size': 128, 'epochs': 200, 'init': 'glorot_uniform', 'optimizer': 'sgd'}
0.849500 (0.008631) with: {'batch_size': 16, 'epochs': 20, 'init': 'glorot_uniform', 'optimizer': 'sgd'}
0.846100 (0.009118) with: {'batch_size': 16, 'epochs': 20, 'init': 'glorot_uniform', 'optimizer': 'rmsprop'}
0.848100 (0.009447) with: {'batch_size': 16, 'epochs': 20, 'init': 'glorot_normal', 'optimizer': 'sgd'}
0.849400 (0.008952) with: {'batch_size': 16, 'epochs': 20, 'init': 'glorot_normal', 'optimizer': 'rmsprop'}
0.843600 (0.010627) with: {'batch_size': 16, 'epochs': 200, 'init': 'glorot_uniform', 'optimizer': 'sgd'}
0.847100 (0.011905) with: {'batch_size': 16, 'epochs': 200, 'init': 'glorot_uniform', 'optimizer': 'rmsprop'}
0.846800 (0.007019) with: {'batch_size': 16, 'epochs': 200, 'init': 'glorot_normal', 'optimizer': 'sgd'}
0.845200 (0.010980) with: {'batch_size': 16, 'epochs': 200, 'init': 'glorot_normal', 'optimizer': 'rmsprop'}
0.841100 (0.009521) with: {'batch_size': 1

**TODO**: how to pass callable OCC.loss to CV?

### 2. Integer encoding

Compare cross entropy vs mse; output dim = 1 and output activation is relu (or sigmoid)

**TODO**: create pipeline that first compiles and tests model with categorical encoding, then compiles and tests model with integer encoding

In [ ]:
model_int = KerasClassifier(build_fn=create_model, p=P, k=1, output_act='relu', verbose=0)

grid_int = GridSearchCV(estimator=model_int, param_grid=param_grid, cv=n_folds)

# integer encoding
grid_int_result = grid_int.fit(X, y)

### 3. Sparse integer encoding

Dimension of output layer is K, activation is relu?

In [ ]:
# ordinal crossentropy loss
model.compile(loss=OCC.loss,
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(train_x, train_y_cat, epochs=20, batch_size=128)
score = model.evaluate(test_x, test_y_cat, batch_size=128)
score

## Observations

- Sparse integer encoding is actually the worst!
- Accuracy about the same for cross_entropy and ordinal_cross_entropy loss functions + categorical encoding

This is still using the default network settings